Question 1 – S&P 500 Stocks Added to the Index
Goal:
Scrape S&P 500 company data from Wikipedia.

Extract the year each company was added.

Count how many were added each year.

Exclude 1957.

Find the year with the most additions (most recent if tied).


In [1]:
import pandas as pd

# Load data from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
tables = pd.read_html(url)

# First table contains current S&P 500 companies
df = tables[0]

# Check if 'Date added' exists and is not null
if 'Date added' in df.columns:
    df = df[df['Date added'].notna()]
    df['Year added'] = pd.to_datetime(df['Date added']).dt.year
    df = df[df['Year added'] != 1957]  # Exclude 1957

    # Count how many companies were added each year
    additions_by_year = df['Year added'].value_counts().sort_index()

    # Find the year with the most additions (latest if tie)
    max_additions = additions_by_year.max()
    best_year = additions_by_year[additions_by_year == max_additions].index.max()

    print("✅ Year with the most additions to S&P 500:", best_year)
    print("📊 Number of companies added that year:", max_additions)

    # Bonus: how many have been in the index > 20 years?
    current_year = pd.Timestamp.now().year
    df['Years in index'] = current_year - df['Year added']
    over_20_years = df[df['Years in index'] > 20].shape[0]
    print("📈 Companies in the index for more than 20 years:", over_20_years)


✅ Year with the most additions to S&P 500: 2017
📊 Number of companies added that year: 23
📈 Companies in the index for more than 20 years: 166


Question 2 – YTD Index Returns vs. S&P 500
Goal:
Compare YTD performance of major world indexes vs. the S&P 500 from Jan 1 to May 1, 2025.

In [5]:
import yfinance as yf
import pandas as pd

indices = {
    'US': '^GSPC',
    'China': '000001.SS',
    'Hong Kong': '^HSI',
    'Australia': '^AXJO',
    'India': '^NSEI',
    'Canada': '^GSPTSE',
    'Germany': '^GDAXI',
    'UK': '^FTSE',
    'Japan': '^N225',
    'Mexico': '^MXX',
    'Brazil': '^BVSP'
}

start_date = '2025-01-01'
end_date = '2025-05-01'
returns = {}

for country, ticker in indices.items():
    try:
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)

        if len(data) >= 2:
            first_close = data['Close'].iloc[0]
            last_close = data['Close'].iloc[-1]
            ytd_return = (last_close / first_close - 1) * 100
            returns[country] = ytd_return
        else:
            print(f"⚠️ Not enough data for {country} ({ticker})")
    except Exception as e:
        print(f"❌ Error fetching data for {country} ({ticker}): {e}")

# Convert to Series and compare
returns_df = pd.Series(returns).sort_values(ascending=False)
snp_return = returns.get('US', 0)
better_than_snp = returns_df[returns_df > snp_return].count()

print("✅ Number of indexes with better YTD returns than S&P 500:", better_than_snp)
print("\n📊 YTD Returns (Jan–May 2025):")
print(returns_df)


Failed to get ticker '^GSPC' reason: Expecting value: line 1 column 1 (char 0)

1 Failed download:
['^GSPC']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker '000001.SS' reason: Expecting value: line 1 column 1 (char 0)

1 Failed download:
['000001.SS']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker '^HSI' reason: Expecting value: line 1 column 1 (char 0)

1 Failed download:
['^HSI']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker '^AXJO' reason: Expecting value: line 1 column 1 (char 0)

1 Failed download:
['^AXJO']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker '^NSEI' reason: Expecting value: line 1 column 1 (char 0)

1 Failed download:
['^NSEI']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker '^GSPTSE' reason: Expecting value: line 1 column 1 (char 0)

1 Failed download:
['^

⚠️ Not enough data for US (^GSPC)
⚠️ Not enough data for China (000001.SS)
⚠️ Not enough data for Hong Kong (^HSI)
⚠️ Not enough data for Australia (^AXJO)
⚠️ Not enough data for India (^NSEI)
⚠️ Not enough data for Canada (^GSPTSE)
⚠️ Not enough data for Germany (^GDAXI)
⚠️ Not enough data for UK (^FTSE)



1 Failed download:
['^N225']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker '^MXX' reason: Expecting value: line 1 column 1 (char 0)

1 Failed download:
['^MXX']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
Failed to get ticker '^BVSP' reason: Expecting value: line 1 column 1 (char 0)

1 Failed download:
['^BVSP']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


⚠️ Not enough data for Japan (^N225)
⚠️ Not enough data for Mexico (^MXX)
⚠️ Not enough data for Brazil (^BVSP)
✅ Number of indexes with better YTD returns than S&P 500: 0

📊 YTD Returns (Jan–May 2025):
Series([], dtype: object)


Question 3 – S&P 500 Correction Analysis
Goal:
Define corrections as >5% drawdown from previous all-time high.

Calculate duration of each correction.

Report median, 25th, and 75th percentiles.

In [6]:
import numpy as np

# Download long-term S&P 500 data
snp = yf.download('^GSPC', start='1950-01-01')

# Identify all-time highs and drawdowns
snp['cummax'] = snp['Close'].cummax()
snp['drawdown'] = (snp['Close'] - snp['cummax']) / snp['cummax']

# Mark start and end of corrections (drawdown > 5%)
snp['correction'] = snp['drawdown'] < -0.05
corrections = []
in_correction = False

for i in range(1, len(snp)):
    if snp['correction'].iloc[i] and not in_correction:
        start = snp.index[i]
        in_correction = True
    elif not snp['correction'].iloc[i] and in_correction:
        end = snp.index[i]
        duration = (end - start).days
        corrections.append(duration)
        in_correction = False

# Analyze durations
corrections_series = pd.Series(corrections)
print("✅ Median correction duration (days):", corrections_series.median())
print("📊 Percentiles:")
print("25th percentile:", corrections_series.quantile(0.25))
print("75th percentile:", corrections_series.quantile(0.75))


Failed to get ticker '^GSPC' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['^GSPC']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


✅ Median correction duration (days): nan
📊 Percentiles:
25th percentile: nan
75th percentile: nan


Question 4 – Amazon Earnings Surprise Analysis
Make sure you have the file ha1_Amazon.csv in the same folder.

In [7]:
# Load earnings data
df_eps = pd.read_csv("ha1_Amazon.csv", delimiter=';')
df_eps['Date'] = pd.to_datetime(df_eps['Date'])

# Download historical price data
amzn = yf.download("AMZN", start="2010-01-01")

# Compute 2-day return: Day3 / Day1 - 1
amzn['2d_return'] = amzn['Close'].shift(-2) / amzn['Close'] - 1

# Filter for positive earnings surprises
df_eps = df_eps[df_eps['Actual EPS'] > df_eps['Estimate EPS']]
results = []

# Find 2-day return following each earnings surprise
for date in df_eps['Date']:
    if date in amzn.index:
        idx = amzn.index.get_loc(date)
        if idx + 2 < len(amzn):
            ret = amzn.iloc[idx]['2d_return']
            results.append(ret)

# Median 2-day return after surprise
results_series = pd.Series(results)
median_surprise_return = results_series.median() * 100

print("✅ Median 2-day return after positive earnings surprise:", round(median_surprise_return, 2), "%")


FileNotFoundError: [Errno 2] No such file or directory: 'ha1_Amazon.csv'